Author: Jakub Gnyp; contact: gnyp.jakub@gmail.com, LinkedIn: https://www.linkedin.com/in/gnypit/
Functions and constants for hashing based on FIPS PUB 180-4 from:
http://dx.doi.org/10.6028/NIST.FIPS.180-4

For a summary of applied knowledge I refer to "Applied Quantum Cryptography" (Lecture notes in physics 797) published by Springer.

In [24]:
import ast
import random
import math
import numpy as np
from scipy.stats import binom

random.seed(a=261135)  # useful for debugging/presentations, should be omitted for practical uses

# Defining functions for privacy amplification

In [25]:
def constant_sha1(index):
    if 0 <= index <= 19:
        return 0x5a827999
    elif 20 <= index <= 39:
        return 0x6eD9eba1
    elif 40 <= index <= 59:
        return 0x8f1bbcdc
    elif 60 <= index <= 79:
        return 0xca62c1d6
    else:
        return 'Error: index exceeds allowed range.'


def ch(x, y, z):  # one of internal hashing functions for SHA-1
    return (x & y) ^ (~x & z)


def parity(x, y, z):  # one of internal hashing functions for SHA-1
    return x ^ y ^ z


def maj(x, y, z):  # one of internal hashing functions for SHA-1
    return (x & y) ^ (x & z) ^ (y & z)


def hashing_function_sha1(x, y, z, j):
    if 0 <= j <= 19:
        return ch(x=x, y=y, z=z)
    elif 20 <= j <= 39 or 60 <= j <= 79:
        return parity(x=x, y=y, z=z)
    elif 40 <= j <= 59:
        return maj(x=x, y=y, z=z)
    else:
        return 'Error: j exceeds possible range.'


def rotl(n, n_bits, word):  # word will be given as an int, so we will return it as an int too
    """python's left shift doesn't discard any bits, just adds padding - we will discard bits manually"""
    first = bin(word << n)[2 + n:n_bits + n + 2]  # bin changes type to str
    first = int('0b' + first, base=2)

    """python's right shift only discards the bits - we will add the padding manually"""
    second = bin(word >> n_bits - n)[2:n + 2]  # bin changes type to str
    second = '0' * (n_bits - n) + second
    second = int('0b' + second, base=2)

    """Result is a bitwise OR on first and second"""
    result = first | second  # result = (word << n) | (word >> (n_bits - n)) with proper discarding and padding
    return result


def sha1(message):
    # message = str(input('Please give a message:'))  # should be a series of 0's and 1's

    """SHA-1: firstly we parse the message into equally long substrings - blocks:"""
    message_length, blocks_length = len(message), 512
    blocks = [message[i:i + blocks_length] for i in range(0, message_length, blocks_length)]
    if len(blocks[-1]) < blocks_length:  # in case the last block is too short, we add padding with 0's
        padding_length = blocks_length - len(blocks[-1])
        padding = '0' * padding_length
        blocks[-1] = padding + blocks[-1]

    """Now for the actual hash computation. We will use:
    words -> groups 32 bits
    hashing function -> as defined above for SHA-1
    ROTL function -> as defined above
    ...to create a 'message schedule' for every block of parsed message from the list 'blocks':
    """

    """SHA-1: setting initial hash value"""
    hash_value = [
        0x67452301,
        0xefcdab89,
        0x98badcfe,
        0x10325476,
        0xc3d2e1f0
    ]

    for block in blocks:
        """Firstly we prepare the message schedule:"""
        for t in range(0, 80, 1):
            """First 16 words of the message schedule are words of the block of parsed message which we are working on
            in this iteration of the loop.
            """
            message_schedule = [block[i:i + 32] for i in range(0, 512, 32)]

            """Last 64 words of the message schedule are created based on a recurrent formula using ROTL function:"""
            for i in range(16, 80, 1):
                """We locally represent selected previous words as binary values"""
                schedule_i3 = int('0b' + message_schedule[i - 3], base=2)
                schedule_i8 = int('0b' + message_schedule[i - 8], base=2)
                schedule_i14 = int('0b' + message_schedule[i - 14], base=2)
                schedule_i16 = int('0b' + message_schedule[i - 16], base=2)

                """Next word of the message schedule is value of ROTL on above words:"""
                next_word = bin(rotl(
                    n=1,
                    n_bits=32,
                    word=schedule_i3 ^ schedule_i8 ^ schedule_i14 ^ schedule_i16
                ))[2:32]
                message_schedule.append(next_word)

            """We initialize the five working variables with last (possibly initial) hash value"""
            a = hash_value[0]
            b = hash_value[1]
            c = hash_value[2]
            d = hash_value[3]
            e = hash_value[4]

            """We process the working variables..."""
            for i in range(0, 80, 1):
                t = rotl(n=5, n_bits=32, word=a) + hashing_function_sha1(x=b, y=c, z=d, j=i) + e + constant_sha1(index=i) \
                    + int('0b' + message_schedule[i], base=2)
                e = d
                d = c
                c = rotl(n=30, n_bits=32, word=b)
                b = a
                a = t

            """...and compute the next hash value.
            As we always need just the last hash value, we won't store all of them in memory; instead we will
            save each next one in the list hash_value, which decomposes such a value
            into its' five words - the list elements. 
            """
            hash_value[0] = hash_value[0] + a
            hash_value[1] = hash_value[1] + b
            hash_value[2] = hash_value[2] + c
            hash_value[3] = hash_value[3] + d
            hash_value[4] = hash_value[4] + e

    """After the loop the resulting 160-bit message digest of the original message is:"""
    digest = ''
    for value in hash_value:
        word = bin(value)
        digest += word[2:len(word)]

    return digest

# Defining function for BINARY:

In [26]:
def binary(sender_block, receiver_block, indexes, receiver_name='Bob'):
    is_binary = True
    sender_current_block = sender_block
    receiver_current_block = receiver_block
    indexes = list(indexes)

    """As in real-life applications, in this simulation of the BINARY algorithm Alice (sender) and Bob (receiver) 
    exchange messages. Additionally, we count how many bits are exchanged between them for the future
    privacy amplification. Afterwards we return this value together with the bit to be changed as a result of the
    algorithm.
    """
    bit_counter = 0

    while is_binary:
        """Sender starts by sending to the Receiver parity of the first half of her string"""
        half_index = len(sender_current_block.values()) // 2  # same as Bob's
        first_half_indexes = indexes[0:half_index:1]  # same as Bob's
        sender_first_half_list = []

        for index in first_half_indexes:
            sender_first_half_list.append(int(sender_current_block.get(index)))

        sender_first_half_parity = sum(sender_first_half_list) % 2
        print("[Alice] My string's first half has a parity: {}".format(sender_first_half_parity))
        bit_counter += 1  # At this point sender informs receiver about their 1st half's parity

        """Now Receiver determines whether an odd number of errors occurred in the first or in the
        second half by testing the parity of his string and comparing it to the parity sent
        by Sender
        """

        receiver_first_half_list = []

        for index in first_half_indexes:
            receiver_first_half_list.append(int(receiver_current_block.get(index)))

        receiver_first_half_parity = sum(receiver_first_half_list) % 2

        """Single (at least) error is in the 'half' of a different parity; we change current strings
        that are analysed into halves of different parities until one bit is left - the error
        """

        if receiver_first_half_parity != sender_first_half_parity:
            print('[{}] I have an odd number of errors in my first half.'.format(
                receiver_name
            ))
            bit_counter += 1

            sender_subscription_block = {}
            receiver_subscription_block = {}

            for index in first_half_indexes:
                receiver_subscription_block[index] = receiver_current_block.get(index)
                sender_subscription_block[index] = sender_current_block.get(index)

            sender_current_block = sender_subscription_block
            receiver_current_block = receiver_subscription_block

            indexes = list(sender_current_block.keys())  # same as Bob's
            
            print('Alice and Bob updated their strings.')
        else:
            print('[{}] I have an odd number of errors in my second half.'.format(
                receiver_name
            ))
            bit_counter += 1

            """We have to repeat the whole procedure for the second halves"""
            second_half_indexes = indexes[half_index::1]
            sender_subscription_block = {}
            receiver_subscription_block = {}

            for index in second_half_indexes:
                receiver_subscription_block[index] = receiver_current_block.get(index)
                sender_subscription_block[index] = sender_current_block.get(index)

            sender_current_block = sender_subscription_block
            receiver_current_block = receiver_subscription_block

            indexes = list(sender_current_block.keys())  # same as Bob's

            print('Alice and Bob updated their strings.')

        if len(receiver_current_block) == 1:  # at some point this clause will be true
            print("[{}] I have one bit left, I'm changing it.".format(
                receiver_name
            ))
            bit_counter += 1  # At this point receiver would send a message (?) about one bit left and changing it

            """We return the correct value of the bit which was erroneous, together with it's index in the original Bob's
            bits string & the number of bits exchanged during BINARY.
            """
            if receiver_current_block[indexes[0]] == '0':
                return ['1', indexes[0], bit_counter]
            else:
                return ['0', indexes[0], bit_counter]

# Performing BB84

Let's set up the quantum channel (BB84):

In [27]:
basis_mapping = {'rectilinear': 0, 'diagonal': 1}
states_mapping = {'0': 0, '1': 1, '+': 0, '-': 1}
quantum_channel = {
    'rectilinear': {
        'basis_vectors': {'first_state': '0', 'second_state': '1'}
    },
    'diagonal': {
        'basis_vectors': {'first_state': '+', 'second_state': '-'}
    }
}

## Let's define some useful functions for the protocol to run:

In [28]:
def qc_gain(mean_photon_number=1., fiber_loss=1., detection_efficiency=1., k_dead=1.,
            additional_loss=1.):  # quantum channel gain -> do lamusa
    g = mean_photon_number * fiber_loss * detection_efficiency * k_dead * additional_loss
    return g


def received_key_material(quantum_channel_gain, sender_data_rate):
    receiver = quantum_channel_gain * sender_data_rate
    return receiver


def random_choice(length, p=0.5):  # function for random choosing of basis for each photon
    """p -> probability of selecting rectilinear basis"""
    chosen_basis = ''
    for index in range(int(np.floor(length))):
        basis = random.uniform(0, 1)
        if basis <= p:
            chosen_basis += str(0)
        else:
            chosen_basis += str(1)

    return chosen_basis


def measurement(state, basis):  # meant for classical encoding

    if basis == '1':  # meaning diagonal basis
        basis = 'diagonal'

        if state == '+':
            final_state = '+'
        elif state == '-':
            final_state = '-'
        elif state == '0' or state == '1':  # in this case there's a 50% chance of getting either polarization
            if random.randint(0, 1) == 0:
                final_state = quantum_channel.get(basis).get('basis_vectors').get('first_state')
            else:
                final_state = quantum_channel.get(basis).get('basis_vectors').get('second_state')
        else:
            return 'U'  # U for unknown

    elif basis == '0':  # meaning rectilinear basis

        if state == '0':
            final_state = '0'
        elif state == '1':
            final_state = '1'
        elif state == '+' or state == '-':
            final_state = str(random.randint(0, 1))  # since '0' and '1' are states, there's no need for if...else
        else:
            return 'U'  # U for unknown

    elif basis == 'L':
        final_state = 'L'  # L for loss, as basis L reflects unperformed measurement due to quantum channel loss
    else:
        return 'U'  # U for unknown

    return final_state


def numerical_error_prob(n_errors, pass_size, qber):  # probability that 2*n_errors remain
    prob = binom.pmf(2 * n_errors, pass_size, qber) + binom.pmf(2 * n_errors + 1, pass_size, qber)
    return prob


def cascade_blocks_sizes(quantum_bit_error_rate, key_length, n_passes=1):
    print('--------------------\nCOMPUTING SIZES OF BLOCKS FOR CASCADE')
    max_expected_value = -1 * math.log(0.5, math.e)
    # best_expected_value = max_expected_value
    best_size = 0
    print("Currently the greatest size of block fulfilling both criteria is {}.".format(best_size))

    for size in range(key_length // 2):  # we need at lest 2 blocks to begin with
        # Firstly we check condition for expected values
        expected_value = 0

        for j in range(size // 2):
            expected_value += 2 * (j + 1) * numerical_error_prob(n_errors=(j + 1), pass_size=size,
                                                                 qber=quantum_bit_error_rate)

        if expected_value <= max_expected_value:
            first_condition = True
        else:
            first_condition = False

        # Secondly we check condition for probabilities per se
        second_condition = False
        for j in range(size // 2):
            prob_sum = 0
            for k in list(np.arange(j + 1, size // 2 + 1, 1)):
                prob_sum += numerical_error_prob(n_errors=k, pass_size=size, qber=quantum_bit_error_rate)

            if prob_sum <= numerical_error_prob(n_errors=j, pass_size=size, qber=quantum_bit_error_rate) / 4:
                second_condition = True
            else:
                second_condition = False

        if first_condition and second_condition:
            if size > best_size:
                # best_expected_value = expected_value
                best_size = size
                print("Currently the greatest size of block fulfilling both criteria is {}.".format(best_size))

    sizes = [best_size]

    for j in range(n_passes - 1):  # corrected interpretation of number of passes
        next_size = 2 * sizes[-1]
        if next_size <= key_length:
            sizes.append(next_size)
        else:
            break

    print("Since the best initial size of blocks equals {} and we are ment to perform {} passes of the algorithm,"
          "sizes of blocks for consecutive passes are: {}".format(best_size, n_passes, sizes))

    return sizes


def cascade_blocks_generator(string_length, blocks_size):
    string_index = list(np.arange(0, string_length, 1))  # I create a list of all indexes (list of ndarray)
    blocks = random.sample(population=string_index, k=string_length)  # I shuffle the list randomly

    for j in range(0, string_length, blocks_size):  # I generate equally long chunks of shuffled indexes
        yield blocks[j:j + blocks_size]


def naive_error(alice_key, bob_key, publication_prob_rect):
    """
    Originally I've been testing algorithm on short strings, i.e. up to 20 bits in original Alice's message.
    Sifted keys were therefore too short for publication of any parts of them for error estimation.

    A refined estimation is based on two subsets: one of the sifted key and the other on the bits originating from
    measurements with different basis (Alice's and Bob's). First we present the 'naive' estimation.

    Let's randomly publish subsets of bits of matching positions in the sifted keys strings. Then we count how many
    bits differ between these two strings, divide this by their length and get a naive estimator of the total error
    rate (between error correction algorithm, i.e. CASCADE).

    As a result of this function we return the naive estimator and Alice's & Bob's keys without published bits.
    """
    print('----------------------------------------\nPERFORMING NAIVE ERROR ESTIMATION\n')
    alice_published_bits = ''
    alice_sifted_key_after_publication = ''
    bob_published_bits = ''
    bob_sifted_key_after_publication = ''

    naive_error_estimate = 0

    for index in range(len(alice_key)):  # could be bob_key as well
        if random.uniform(0, 1) <= publication_prob_rect:
            alice_bit = alice_key[index]
            bob_bit = bob_key[index]

            """First we add those bits to strings meant for publication"""
            alice_published_bits += alice_bit
            bob_published_bits += bob_bit

            """Now for the estimation of the error:"""
            if alice_bit != bob_bit:
                naive_error_estimate += 1
        else:  # if a bit wasn't published, we reuse it in the sifted key
            alice_sifted_key_after_publication += alice_key[index]
            bob_sifted_key_after_publication += bob_key[index]

    print('[Alice] My subset of bits for error estimation is: {}'.format(alice_published_bits))
    print('[Bob] My subset of bits for error estimation is: {}'.format(bob_published_bits))

    try:
        naive_error_estimate = naive_error_estimate / len(alice_published_bits)
    except ZeroDivisionError:
        naive_error_estimate = 0  # this will obviously be false, but easy to notice and work on in genalqkd.py

    """At this point we count the number of bits exchanged between Alice and Bob via the public channel,
    for future estimation of the computational cost."""

    no_published_bits = len(alice_published_bits) + len(bob_published_bits)
    print('The probability of bit publishing was {}. In actuality {} bits were published out of {} bits succesfully '
          'sent & received.'.format(publication_prob_rect, no_published_bits, len(alice_key)))

    results = {
        'error estimator': naive_error_estimate,
        'alice key': alice_sifted_key_after_publication,
        'bob key': bob_sifted_key_after_publication,
        'number of published bits': no_published_bits
    }

    return results


def refined_average_error(rect_prob, rect_pub_prob, diag_pub_prob, alice_key, bob_key, alice_bases, bob_bases):
    """In the refined error analysis for simplicity we DO NOT divide raw keys into two separate strings (by the basis).
    Instead, we create two empty strings - alice_key_after_error_estimation & bob_key_after_error_estimation - into
    which we shall rewrite bits unused for error estimation. As for the others, chosen with probability
    rect_pub_prob & diag_pub_prob, respectively, we count them as 'published' and additionally count an error,
    if they differ from each other. Those counter are:
    rect_pub_counter & diag_pub_counter, rect_error & diag_error.

    The last two will be divided at the end by the first two, respectively, to obtain estimations as ratios.
    """
    print('----------------------------------------\nPERFORMING REFINED ERROR ESTIMATION\n')
    alice_key_after_error_estimation = ''
    bob_key_after_error_estimation = ''
    rect_error = 0
    rect_pub_counter = 0
    diag_error = 0
    diag_pub_counter = 0
    alice_published_bits_rect = ''
    bob_published_bits_rect = ''
    alice_published_bits_diag = ''
    bob_published_bits_diag = ''

    """By the way, we don't have to worry with the conditional probability, because the final formula for the error
    estimation takes that into consideration.
    """
    for index in range(len(alice_key)):
        if alice_bases[index] == bob_bases[index] == '0':  # rectilinear basis
            if random.uniform(0, 1) >= rect_pub_prob:
                alice_key_after_error_estimation += alice_key[index]
                bob_key_after_error_estimation += bob_key[index]
            else:
                rect_pub_counter += 1
                if alice_key[index] != bob_key[index]:
                    rect_error += 1
                alice_published_bits_rect += alice_key[index]
                bob_published_bits_rect += bob_key[index]
        else:
            if random.uniform(0, 1) >= diag_pub_prob:
                alice_key_after_error_estimation += alice_key[index]
                bob_key_after_error_estimation += bob_key[index]
            else:
                diag_pub_counter += 1
                if alice_key[index] != bob_key[index]:
                    diag_error += 1
                alice_published_bits_diag += alice_key[index]
                bob_published_bits_diag += bob_key[index]

    """As it is possible to get a VERY small probability of publication, we check for possible divisions by zero:"""
    try:
        rect_error = float(rect_error) / float(rect_pub_counter)
    except ZeroDivisionError:
        rect_error = 0.0

    try:
        diag_error = float(diag_error) / float(diag_pub_counter)
    except ZeroDivisionError:
        diag_error = 0.0

    print('[Alice] My subset of bits measured in the rectilinear basis is: {}'.format(alice_published_bits_rect))
    print('[Bob] My subset of bits measured in the rectilinear basis is: {}'.format(bob_published_bits_rect))
    print('The error rate estimate for the rectilinear basis is {}.\n'.format(rect_error))

    print('[Alice] My subset of bits measured in the diagonal basis is: {}'.format(alice_published_bits_diag))
    print('[Bob] My subset of bits measured in the diagonal basis is: {}'.format(bob_published_bits_diag))
    print('The error rate estimate for the diagonal basis is {}.\n'.format(diag_error))

    """Now, given that measurements in the rectilinear basis were not necessarily with the same probability 
    as those in the diagonal basis, we need a more complicated formula for the 'average error estimate' 
    (Lo, Chau, Ardehali, 2004).
    """
    p = rect_prob  # just a reminder that it's the probability of choosing rect. basis for measurements
    e1 = rect_error
    e2 = diag_error

    e = (p ** 2 * e1 + (1 - p) ** 2 * e2) / (p ** 2 + (1 - p) ** 2)
    print('The average error estimate is {}.'.format(e))

    results = {
        'error estimator': e,
        'alice key': alice_key_after_error_estimation,
        'bob key': bob_key_after_error_estimation,
        'number of published bits': rect_pub_counter + diag_pub_counter
    }

    return results

## Start of the protocol (input section)

At the begining we define the inputs, namely: quantum gain of a quantum channel used by Alice and Bob, Alice's basis length, etc.

In [29]:
"""Please enter a numerical value between 0 and 1, e.g. 0.95:"""
gain = 0.95

"""Alice will choose her basis randomly. How long should the string with her choices be?"""
alice_basis_length = 512

"""Please set the Alice's probability of choosing a rectilinear basis:"""
rectilinear_basis_prob = 0.75

"""There's a probability that due to disturbances in quantum channel or eavesdropping some bits change while being sent"""
change_probability = 0.05

"""Please set the number of CASCADE passes to perform:"""
cascade_passes = 4

At this point the quantum channel is set up and we know how Alice prepares the measurements.
It's time to perform sending states by Alice. She will choose tem randomly, with unifrom probability.

In [30]:
if alice_basis_length != 0:  # firstly it was 0, if now it's !=0, then we are supposed to choose randomly her basis
    alice_basis = random_choice(alice_basis_length, p=rectilinear_basis_prob)
    print('Random choices of basis for Alice are: {}'.format(alice_basis))

alice_bits_length = 0
if alice_bits_length == 0:  # if it's 0, then we didn't get the bits form the user
    alice_bits_length = alice_basis_length
    alice_bits = random_choice(alice_bits_length, p=0.5)
    print('Random choices of bits for Alice are: {}'.format(alice_bits))

Random choices of basis for Alice are: 10000000001110010011001000000010000000000000000000010001100100001001000000000010001001010001000000000100000111100000001100010000000101000000100101000000000001001000000010111001101010000001100100000000010100000000000101110101011000110111001010000100010110111110110001010100000001001010111110100000000000000000000001100010010000000000000001010001000101100011000000000101100001011000000110000000010001100100001100000001000010101101010000110000001010000100000010101101010001001100000000001100110001000101000100000011
Random choices of bits for Alice are: 11010100010110110110111101010101101101000000011001011011010010110010110101011011001100010111000011001010100010110111111001011011010101101001010000111100010100010101111111101010111110101011000001011100101000100001111011010010111110111011011100100001011111000001011110111111010000110011010011000011000000000001110111000111111001010010001101000101010101110001001110000110001110111000000100111100010100101001001110

At this point it is impractical to encode states as bits because Bob's measurements results depend on both basis
and bit choice of Alice, but he shouldn't know the first one. Because of that we will now translate Alice's bits
to proper states, changing 0 and 1 into + and - for the diagonal basis, respectively.

Please remember that we do that in order to demonstrate what exactly is going on - in practice a measurement is 
performed, results are remembered and post-processed and they are not being published, in contrary to us printing
all of the information ;)

In [31]:
i = 0
alice_states_list = list(alice_bits)
for bit in alice_bits:
    if alice_basis[i] == '1':
        if bit == '0':
            alice_states_list[i] = '+'
        elif bit == '1':
            alice_states_list[i] = '-'
        else:
            alice_states_list[i] = 'U'  # U for unknown
    i += 1

alice_states = ''.join(alice_states_list)
print("Alice's states are: {}, where U stands for unknown states due to incorrect encoding of bits.".format(
    alice_states
))

Alice's states are: -101010001+--01-01-+11-1010101+11011010000000110010-101-+10+1011+01+1101010110-100-10+0-011-000011001+10100+-+-1011111-+010-1011010-0-101001+10+0+11110001010+01+1011111-1-+-01+-1-1-010101-+00+010111001+1+00100001111+1-+-0+1+1--110--1+--01-1+0100+010-1--1+++++1+-111+1-1-1101000+11+0-1+-++-1+0001100000000000111011-+001-11-100101001000110-0+010-010-0--100+-001110000-1++0111+1--000000-+01111000-010+-01+0100--1011100+1011+0+1++1+0+0010--101011+1+1100+110001-0-0+-0-0-110-01+-1011001100+-00-+010+110+1-111-001011++, where U stands for unknown states due to incorrect encoding of bits.


Now that we have Alice's data rate, quantum channel's gain and Alice's states,
we can randomly choose m (Alice's basis choices number) bases for Bob. While he performs his measurements,
a portion of Alice's photons do not reach him due to loss on quantum channel.

In [32]:
bob_basis_list = random_choice(alice_basis_length, p=rectilinear_basis_prob)
bob_basis = ''
for basis in bob_basis_list:
    if random.uniform(0, 1) <= gain:  # for small numbers of bits quantum gain won't change anything in here
        bob_basis += str(int(basis))
    else:
        bob_basis += 'L'  # L for loss

print("Bob's basis choices are: {}, where L stands for measurement that cannot be done".format(bob_basis),
      "\ndue to quantum channel loss.")

Bob's basis choices are: 0L00000110000L000001100100000000110001010L10000011101000L0010000101101010010000001100010L1L00L001100000000010010L001100L01001000101000L0000100000010010000000000L00010000000L010000000000110011100101L000000000010LL0001100101001001L000000101L000011000000000L001000000L000101001000001010001010L0000011100000L0000L001000100110000L010001000000000L0000000100100010L110100000000100100010010000001101000000L00101L00100100001001011101111000000001000000100010010001L101100011001001000100L00001000000000000001000010000000010, where L stands for measurement that cannot be done 
due to quantum channel loss.


Now we can perform disturbances in the quantum channel:

In [33]:
received_states = ''
change_states = {'0': '0', '1': '1', '2': '+', '3': '-'}  # dictionary for randomizing changed states
for state in alice_states:  # to be updated following concrete attack strategies
    if random.uniform(0, 1) <= change_probability:
        change_indicator = str(random.randint(0, 3))
        while state == change_states.get(change_indicator):  # we repeat as long as it takes to actually change state
            change_indicator = str(random.randint(0, 3))
        received_states += change_states.get(change_indicator)
    else:
        received_states += state

print("States received by Bob before measurement are: {}".format(received_states))

States received by Bob before measurement are: -101010001+--01001-+11-1010101+11011010000000110010-101-+10+1011+01+110101+110-100-10+0-011-000011001+10100+-+-1011111-+010-1011010-0-10+001+10+0+11110001010+01+1011111-1-+-0-011-+-01010+-+00+-10111001+1+0010+001111+1-+-0+1+1--110--1+--01-1+0100+010-1--1+++++1+-111+1-1-1101100+11+0-1+-++-1+0001100000000000111011-+001-11-100101001000100+0+010-010-0--100+-001111000-1++0-11+11--00000-+01101000-010+-01+0-00--1011100+1011+0+1++1+0+0010--101011+1+1100011+001-0-0+-0-0-110-01+-1011001100+-01-+0+0-110+1-111-0010+1++


After Bob chose a basis and a photon has reached him, he performs his measurement. If for this particular photon 
he chooses the same basis as Alice chose before, his measurement result will be the same - that's because
in reality Bob is choosing polarisators for photons to go through. If photon's polarization is the same as
polarisators, then there's 100% probability of preserving this polarization. This is reflected by the same bit
as before the measurement. Otherwise polarization after measurement is random. This mechanism is implemented in
"measurement" function.

Results of Bob's measurements are states - either 0 for |0> or 1 for |1> or + for |+> or - for |->. If he couldn't
perform the measurement (encoded by basis L) then the result will be encoded in the string by L as well.

In [34]:
bob_states = ''
for i in range(alice_basis_length):  # it's the same length as of Bob's basis choices
    bob_states += measurement(state=received_states[i], basis=bob_basis[i])

print("Bob's measurement results are: {}".format(bob_states))

Bob's measurement results are: 0L01010-+1111L10011++11+01010101--110+0-0L+00110+-+1+011L10+1011+0++1-0-01+110110+-101-1L-L10L00+-001110100+10-1L11--10L0-00+011-1+000L0000+010100+11+0001010001L101+1111111L0-0110000101++00+++11-1+L0011110010+0LL111++00-0+10-11+L001110-0-L1101+-101001011L00+011011L110-0-10+10011+0+010-0+0L00001++-00000L0001L10+110-01-+1010L1+100+000100001L1000101-11+001-0L--1-00001110-11+111-00-001001++1+001010L00-0-L00-11-1110+11-1+++1++++00100100-101011+111+00-110-L+0--000+-00+10-011-10L1001+00110100010111+0111-10001011+0


Now, having Bob's measurement results, we can translate states into bits.

In [35]:
bob_bits = ''
for state in bob_states:
    try:
        bob_bits += str(states_mapping[state])  # we want to use indexing to raise errors for unsuccessful measur.
    except KeyError:
        bob_bits += random_choice(length=1)
        continue

print("Bob's bits are: {}".format(bob_bits))

Bob's bits are: 01010101011111100110011001010101111100010000011001010011010010110000110101011011001101111101000001001110100010110111110101000011110000000000010100011000010100011101011111110010110000101000000011110000111100100001111000010010111010011101011110101101001011000001101101101011001001100001010000000010010000000001110011010110101011010000001000010100010111100011001111000011101110111100100100100100010101001010001111111001111000100000010010011010110111000111011001100001000101011110110010001101000101110011111000101100


## Sifting

Alice and Bob each have a string of bits, which will shortly become a key for cipher.
At this point Alice and Bob can switch to communicating on a public channel. Their first step is to 
perform sifting - decide which bits to keep in their key.
    
Bob begins by telling Alice, which photons he measured. He then tells her which basis were used in each measurement.
Then it's Alice's turn to send Bob her basis and to cancel out bits (representing states!) from her string 
that do not match both successful Bob's measurement and his usage of the same basis in each case.

In [36]:
bob_measurement_indicators = ''
for bit in bob_bits:  # is it possible to optimise length of such an indicator?
    if bit == '0' or bit == '1':
        bob_measurement_indicators += '1'
    else:
        bob_measurement_indicators += '0'

print("[Bob] My measurement indicator: {}".format(bob_measurement_indicators))

bob_indicated_basis = ''
bob_indicated_bits = ''
index = 0

for indicator in bob_measurement_indicators:  # in optimised approach send only bases for successful measurements
    if indicator == '1':
        bob_indicated_basis += bob_basis[index]
        bob_indicated_bits += bob_bits[index]
    index += 1

print("[Bob] I used bases: {}".format(bob_indicated_basis))

[Bob] My measurement indicator: 11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
[Bob] I used bases: 0L00000110000L000001100100000000110001010L10000011101000L0010000101101010010000001100010L1L00L001100000000010010L001100L01001000101000L0000100000010010000000000L00010000000L010000000000110011100101L000000000010LL0001100101001001L000000101L000011000000000L001000000L000101001000001010001010L0000011100000L0000L001000100110000L010001000000000L0000000100100010L110100000000100100010010000001101000000L00101L0010010000100101110111100000000

Now it's Alice's turn to send Bob her bases and to cancel out bits (representing states!) from her string that do not
match both successful Bob's measurement and his usage of the same basis in each case.

In [37]:
alice_indicated_bits = ''
alice_indicated_basis = ''
alice_sifted_key = ''

index = 0
for indicator in bob_measurement_indicators:  # this loop ic practically copied - room for optimisation
    if indicator == '1':
        alice_indicated_bits += alice_bits[index]
        alice_indicated_basis += alice_basis[index]
    index += 1

print("[Alice] I used bases: {}".format(alice_indicated_basis))

index = 0
for basis in alice_indicated_basis:
    if basis == bob_indicated_basis[index]:
        alice_sifted_key += alice_indicated_bits[index]
    index += 1

[Alice] I used bases: 10000000001110010011001000000010000000000000000000010001100100001001000000000010001001010001000000000100000111100000001100010000000101000000100101000000000001001000000010111001101010000001100100000000010100000000000101110101011000110111001010000100010110111110110001010100000001001010111110100000000000000000000001100010010000000000000001010001000101100011000000000101100001011000000110000000010001100100001100000001000010101101010000110000001010000100000010101101010001001100000000001100110001000101000100000011


Now Bob gets info from Alice about her choices of bases, so that he can omit bits resulting from measurements
when he used different basis than Alice.

In [38]:
bob_sifted_key = ''

index = 0
for basis in bob_indicated_basis:
    if basis == alice_indicated_basis[index]:
        bob_sifted_key += bob_indicated_bits[index]
    index += 1

print("Alice's sifted key is {},\n& Bob's sifted key is {}.".format(
    alice_sifted_key, bob_sifted_key
))

Alice's sifted key is 0101011010101010111100000110011001011000100111010110000001101000111110001110010010011000101001101111101101010011001100100111010010110101010110111110000111000010000000011010111101100000110010010001010001011010000011000100001111101100000010110101101100011010101010111010010000010111110010110,
& Bob's sifted key is 0101011010101010111100000110011001011000100111010110000001101000111110001110000010011000101001101111101001010111001100100111010010110101010110111110100111000010000000011010111101100000100010010001010001011010000011000100001111101100000010110101101100111010101010111010010001010111110010110.


## Error estimation

Sifted keys generally differ from each other due to changes between states sent by Alice and received by Bob.
In order to estimate empirical probability of error occurrence in the sifted keys we can publish parts
of keys, compare them and calculate numbers of errors. Then published parts of key should be deleted, as
they have just been exchanged via the public channel.

Originally I've been using 'normal' error estimation; naive one. In this version of the demonstrator we can choose between naive and refined one (Lo, Chau, Ardehali, 2004). For this purpose I defined above two functions: naive_error & refined_average_error.
Therefore, we must choose the method of error estimation (at the begining).

In [39]:
alice_sifted_key_after_error_estimation = ''
bob_sifted_key_after_error_estimation = ''
qber = None
pub_prob_rect = 0.5  # default probability of publication of bits measured in rectilinear basis for refined error est.
pub_prob_diag = 0.5  # default probability of publication of bits measured in diagonal basis for refined error est.
while True:
    print('Please choose a method of error estimation (naive/refined):')
    answer6 = str(input())
    if answer6 not in ('naive', 'refined'):
        print("Sorry, I didn't understand that.")
        continue
    else:
        break

if answer6 == 'naive':
    published_key_length_ratio = 0.2
    print("Default ratio of length of published part of the key to it's full length is 0.2.",
          "\nDo you want to change it ('yes'/'no')?")
    while True:
        answer7 = str(input())
        if answer7 not in ('yes', 'no'):
            print("Sorry, I didn't understand that.")
            continue
        else:
            break

    if answer7 == 'yes':
        print("Please set ratio of number of published bits of the key to it's full length",
              "\n as a numerical value between 0 and 1, e.g. 0.15:")
        while True:
            try:
                published_key_length_ratio = float(input())
                if 0 <= published_key_length_ratio <= 1:
                    break
                else:
                    print("Please enter a numerical value between 0 and 1")
                    continue
            except TypeError:
                print("Please enter a numerical value between 0 and 1")
                continue
    naive_results = naive_error(
        alice_key=alice_sifted_key,
        bob_key=bob_sifted_key,
        publication_prob_rect=published_key_length_ratio)
    qber = naive_results.get('error estimation')
    alice_sifted_key_after_error_estimation = naive_results.get('alice key')
    bob_sifted_key_after_error_estimation = naive_results.get('bob key')
else:  # refined error estimation
    while True:
        try:
            print("Please set the probability of bits' publication in the rectilinear basis:")
            pub_prob_rect = float(input())
        except TypeError:
            print("Please enter a numerical value between 0 and 1, e.g. 0.57.")
            continue
        else:
            break
    while True:
        try:
            print("Please set the probability of bits' publication in the diagonal basis:")
            pub_prob_diag = float(input())
        except TypeError:
            print("Please enter a numerical value between 0 and 1, e.g. 0.57.")
            continue
        else:
            break
    """Now that we have the inputs for the refined error estimation, we perform it with a function defined above."""
    refined_results = refined_average_error(
        rect_prob=rectilinear_basis_prob,
        rect_pub_prob=pub_prob_rect,
        diag_pub_prob=pub_prob_diag,
        alice_key=alice_sifted_key,
        bob_key=bob_sifted_key,
        alice_bases=alice_basis,
        bob_bases=bob_basis
    )
    qber = refined_results.get('error estimator')
    alice_sifted_key_after_error_estimation = refined_results.get('alice key')
    bob_sifted_key_after_error_estimation = refined_results.get('bob key')

Please choose a method of error estimation (naive/refined):
refined
Please set the probability of bits' publication in the rectilinear basis:
0.2
Please set the probability of bits' publication in the diagonal basis:
0.2
----------------------------------------
PERFORMING REFINED ERROR ESTIMATION

[Alice] My subset of bits measured in the rectilinear basis is: 0111100110110100000010000001
[Bob] My subset of bits measured in the rectilinear basis is: 0111100110100101000010000001
The error rate estimate for the rectilinear basis is 0.07142857142857142.

[Alice] My subset of bits measured in the diagonal basis is: 1010101010111101000000100101
[Bob] My subset of bits measured in the diagonal basis is: 1010101010111100000000100101
The error rate estimate for the diagonal basis is 0.03571428571428571.

The average error estimate is 0.06785714285714285.


## CASCADE

Assuming the estimate of empirical probability of error is reasonably small we can continue
with the error correction. Naturally for BB84 we assume it's Bob's key that's flawed.

We begin by checking the parity of Alice's and Bob's sifted keys, 
shortened by the subsets used for error estimation.

CASCADE: Firstly, we need to assign bits to their indexes in original strings. Therefore, we create dictionaries
for Alice and for Bob.

In [40]:
n = len(alice_sifted_key_after_error_estimation)
alice_cascade = {}
bob_cascade = {}

for i in range(n):  # I dynamically create dictionaries with indexes as keys and bits as values
    alice_cascade[str(i)] = alice_sifted_key_after_error_estimation[i]
    bob_cascade[str(i)] = bob_sifted_key_after_error_estimation[i]

Secondly, we need to set up CASCADE itself: sizes of blocks in each pass, numeration of passes and a dictionary
for corrected bits with their indexes from original Bob's string as keys and correct bits as values.

In [41]:
blocks_sizes = cascade_blocks_sizes(quantum_bit_error_rate=qber, key_length=n, n_passes=cascade_passes)
bob_corrected_bits = {}

--------------------
COMPUTING SIZES OF BLOCKS FOR CASCADE
Currently the greatest size of block fulfilling both criteria is 0.
Currently the greatest size of block fulfilling both criteria is 2.
Currently the greatest size of block fulfilling both criteria is 3.
Currently the greatest size of block fulfilling both criteria is 4.
Currently the greatest size of block fulfilling both criteria is 5.
Currently the greatest size of block fulfilling both criteria is 6.
Currently the greatest size of block fulfilling both criteria is 7.
Currently the greatest size of block fulfilling both criteria is 8.
Currently the greatest size of block fulfilling both criteria is 9.
Currently the greatest size of block fulfilling both criteria is 10.
Currently the greatest size of block fulfilling both criteria is 11.
Currently the greatest size of block fulfilling both criteria is 12.
Currently the greatest size of block fulfilling both criteria is 13.
Currently the greatest size of block fulfilling both 

Thirdly, in order to return to blocks from earlier passes of CASCADE we need a history of blocks with indexes and bits,
so implemented by dictionaries as list elements per pass, nested in general history list. Moreover, we create lists
and variables to remember error rates of bits after each consecutive pass of CASCADE and to count the bits exchanged.

In [42]:
history = []
error_rates = []
pass_number = 0
exchanged_bits_counter = 0

for size in blocks_sizes:
    print("--------------------\nThis is CASCADE pass number {}".format(pass_number + 1))
    try:
        pass_number_of_blocks = int(
            -1 * np.floor(-1 * n // size))  # I calculate how many blocks are in total in this pass
    except ZeroDivisionError:
        print("Initial block size equal to 0, please check why.")
        quit()

    print("With {} bits in a single block and {} bits in total we have {} blocks in this CASCADE pass".format(
        size, n, pass_number_of_blocks
    ))

    alice_pass_parity_list = []
    bob_pass_parity_list = []
    alice_blocks = []
    bob_blocks = []

    for block_index in cascade_blocks_generator(string_length=n, blocks_size=size):
        """We sample bits from the raw key (after the error estimation phase) into blocks, with the generator
        defined above.
        """
        alice_block = {}  # a dictionary for a single block for Alice
        bob_block = {}  # a dictionary for a single block for Bob

        for index in block_index:  # I add proper bits to these dictionaries
            alice_block[str(index)] = alice_cascade[str(index)]
            bob_block[str(index)] = bob_cascade[str(index)]

        """I append single blocks created for given indexes to lists of block for this particular CASCADE's pass"""
        alice_blocks.append(alice_block)
        bob_blocks.append(bob_block)

    for i in range(pass_number_of_blocks):
        print(">>>\nThis is block number {} out of {} in CASCADE pass number {}.\n".format(
            i+1, pass_number_of_blocks, pass_number))

        current_indexes = list(alice_blocks[i].keys())  # same as Bob's
        print("Indexes of bits from the raw key which are sampled for the current block in CASCADE are: {}".format(
            current_indexes
        ))

        alice_current_bits = list(alice_blocks[i].values())
        print("Alice's bits with these indexes are: {}".format(alice_current_bits))

        bob_current_bits = list(bob_blocks[i].values())
        print("Bob's bits with these indexes are: {}".format(bob_current_bits))

        alice_bit_values = []
        bob_bit_values = []

        for j in range(len(current_indexes)):
            alice_bit_values.append(int(alice_current_bits[j]))
            bob_bit_values.append(int(bob_current_bits[j]))

        alice_pass_parity_list.append(sum(alice_bit_values) % 2)
        print("[Alice] Bits in my block have parity {}".format(alice_pass_parity_list[-1]))

        bob_pass_parity_list.append(sum(bob_bit_values) % 2)
        print("[Bob] Bits in my block have parity {}".format(bob_pass_parity_list[-1]))

        if alice_pass_parity_list[i] != bob_pass_parity_list[i]:  # we check if we should perform BINARY
            print("[Alice] We have different parities. Let's perform binary search for the erroneous bit.")

            binary_results = binary(
                sender_block=alice_blocks[i],
                receiver_block=bob_blocks[i],
                indexes=current_indexes
            )

            """Firstly we add the number of exchanged bits during this BINARY performance to the general number
            of bits exchanged via the public channel.
            """
            exchanged_bits_counter += binary_results[2]

            """Secondly we change main dictionary with final results and current blocks for history"""
            bob_cascade[binary_results[1]] = binary_results[0]
            bob_blocks[i][binary_results[1]] = binary_results[0]

            """Thirdly we change the error bit in blocks' history
            We need to perform BINARY on all blocks which we correct in history list
            history[number of pass][owner][number of block]
            """
            if pass_number > 0:  # in the first pass of CASCADE there are no previous blocks
                print("[Alice] Since this CASCADE pass number {} we should find blocks "
                      "with the corrected bit from previous passes".format(pass_number))
                for n_pass in range(pass_number):  # we check all previous passes
                    no_blocks = len(history[n_pass - 1][1])
                    for n_block in range(no_blocks):  # we check all Bob's blocks in each previous pass
                        correct_bit_index = binary_results[1]
                        if correct_bit_index in history[n_pass - 1][1][n_block]:
                            """After we found the correct bit's index in one of Bob's blocks from previous pass of the
                            CASCADE, we can correct it and once again perform BINARY on this block (from Alice & Bob).
                            That's because there has to be another erroneous bit, that together with the 'old' error
                            we have just corrected made the parity the same between the blocks.
                            """
                            history[n_pass - 1][1][n_block][correct_bit_index] = binary_results[0]

                            print("[Bob] I found a previous block. Let's go!")
                            print("[Alice] Mee too.")

                            try:
                                if type(history[n_pass - 1][1][n_block]) == str:
                                    indexes = ast.literal_eval(history[n_pass - 1][1][n_block])
                                    binary_previous = binary(
                                        sender_block=history[n_pass - 1][0][n_block],
                                        receiver_block=history[n_pass - 1][1][n_block],
                                        indexes=indexes.keys()
                                    )
                                elif type(history[n_pass - 1][1][n_block]) == dict:
                                    binary_previous = binary(
                                        sender_block=history[n_pass - 1][0][n_block],
                                        receiver_block=history[n_pass - 1][1][n_block],
                                        indexes=history[n_pass - 1][1][n_block].keys()
                                    )
                            except AttributeError:
                                print("AttributeError for binary_previous")

                                file = open("error.txt", "w")
                                file.write('\n' + 'type of history: ' + str(type(history)) + '\n' + 'type of '
                                                                                                    'history['
                                                                                                    'n_pass]: ' +
                                           str(type(history[n_pass])) + '\n' + 'type of history[n_pass][1]: ' +
                                           str(type(history[n_pass][1])) + '\n' + 'type of history[n_pass][1]['
                                                                                  'n_block]: ' +
                                           str(type(history[n_pass][1][n_block])) + '\n' + str(history) + '\n')
                                file.close()
                                exit()

                            exchanged_bits_counter += binary_previous[2]
                            bob_cascade[binary_previous[1]] = binary_previous[0]
                            bob_blocks[i][binary_previous[1]] = binary_previous[0]

    history.append([alice_blocks, bob_blocks])
    pass_number += 1

    """For the purposes of demonstration we check the error rate after each pass:"""
    alice_key_error_check = ''.join(list(alice_cascade.values()))
    bob_key_error_check = ''.join(list(bob_cascade.values()))

    key_error_rate = 0
    index = 0
    for bit in alice_key_error_check:
        if bit != bob_key_error_check[index]:
            key_error_rate += 1
        index += 1
    try:
        key_error_rate = key_error_rate / len(alice_key_error_check)
        error_rates.append(key_error_rate)  # its length is equivalent to no. CASCADE passes performed
    except ZeroDivisionError:
        print("ZeroDivisionError in key error rate calculation.")
        exit()
    print("After this CASCADE pass the ACTUAL error rate is equal to {}.".format(key_error_rate))

--------------------
This is CASCADE pass number 1
With 16 bits in a single block and 233 bits in total we have 15 blocks in this CASCADE pass
>>>
This is block number 1 out of 15 in CASCADE pass number 0.

Indexes of bits from the raw key which are sampled for the current block in CASCADE are: ['197', '50', '92', '88', '119', '191', '146', '19', '85', '84', '128', '33', '9', '45', '53', '219']
Alice's bits with these indexes are: ['1', '1', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1']
Bob's bits with these indexes are: ['1', '1', '1', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1']
[Alice] Bits in my block have parity 0
[Bob] Bits in my block have parity 0
>>>
This is block number 2 out of 15 in CASCADE pass number 0.

Indexes of bits from the raw key which are sampled for the current block in CASCADE are: ['98', '180', '117', '189', '107', '24', '75', '29', '228', '2', '160', '68', '123', '27', '173', '212']
Alice's bits with these indexes ar

Time to create strings from cascade dictionaries into corrected keys:

In [43]:
alice_correct_key = ''.join(list(alice_cascade.values()))
bob_correct_key = ''.join(list(bob_cascade.values()))

print("Alice's correct key:", "\n{}".format(alice_correct_key))
print("Bob's key after performing CASCADE error correction:", "\n{}".format(bob_correct_key))
print("Number of bits exchanged during error correction: {}".format(exchanged_bits_counter))

print("History:", "\n{}".format(history))

Alice's correct key: 
01010010110111100000110011001011000001101010000110100011110011100100011000100111101010101101100001110100101101001110111100011100001000000110011111000010100100101000111010000100100001111101100001011101101101110101010111010000111101010
Bob's key after performing CASCADE error correction: 
01010010110111100000110011001011000001101010000110100011110011100100011000100111101010101101100001110100101101001110111100011100001000000110011111000010100100101000111010000100100001111101100001011101101101110101010111010000111101010
Number of bits exchanged during error correction: 36
History: 
[[[{'197': '1', '50': '1', '92': '1', '88': '1', '119': '1', '191': '0', '146': '0', '19': '0', '85': '0', '84': '1', '128': '0', '33': '0', '9': '1', '45': '0', '53': '0', '219': '1'}, {'98': '1', '180': '0', '117': '1', '189': '0', '107': '1', '24': '1', '75': '0', '29': '0', '228': '0', '2': '0', '160': '1', '68': '0', '123': '1', '27': '0', '173': '1', '212': '0'}, {'64': '0', '73': '0'

All that remains is to randomly choose number of bits for deletion, equal to number of exchanged bits
during error correction phase. It's a form of a rudimentary privacy amplification. Let's say Alice randomly deletes 
bits and informs Bob which indexes they were on, so that the computational cost would be equal 
to the number of deleted bits.

In [44]:
deleted_bits_counter = 0
try:
    deletion_prob = exchanged_bits_counter / len(alice_correct_key)
except ZeroDivisionError:
    print('Error in bit deletion.')
    deletion_prob = 0  # no idea how to set it better in such a case

index = 0
while deleted_bits_counter < exchanged_bits_counter:
    if index == len(alice_correct_key):  # just in case we won't delete enough bits in the first 'run'
        index = 0
    if random.uniform(0, 1) <= deletion_prob:  # we "increase" the prob. by < OR =
        alice_correct_key = alice_correct_key[0: index:] + alice_correct_key[index + 1::]
        bob_correct_key = bob_correct_key[0: index] + bob_correct_key[index + 1::]
        deleted_bits_counter += 1

    index += 1

Now we finally have the proper keys.

In [45]:
print("Alice's correct key:", "\n{}".format(alice_correct_key))
print("Bob's key after performing CASCADE error correction:", "\n{}".format(bob_correct_key))
print("Number of bits exchanged during error correction: {}".format(exchanged_bits_counter))

print("History:", "\n{}".format(history))

Alice's correct key: 
10100101101110000010011001011000001111000011010001111011100100110010111101010011110000111000110100110111000110001000001100111000101010011001110100010000001111101000101110101111010101110000011101010
Bob's key after performing CASCADE error correction: 
10100101101110000010011001011000001111000011010001111011100100110010111101010011110000111000110100110111000110001000001100111000101010011001110100010000001111101000101110101111010101110000011101010
Number of bits exchanged during error correction: 36
History: 
[[[{'197': '1', '50': '1', '92': '1', '88': '1', '119': '1', '191': '0', '146': '0', '19': '0', '85': '0', '84': '1', '128': '0', '33': '0', '9': '1', '45': '0', '53': '0', '219': '1'}, {'98': '1', '180': '0', '117': '1', '189': '0', '107': '1', '24': '1', '75': '0', '29': '0', '228': '0', '2': '0', '160': '1', '68': '0', '123': '1', '27': '0', '173': '1', '212': '0'}, {'64': '0', '73': '0', '150': '1', '230': '0', '227': '1', '13': '1', '167': '0', '122': '0',

## Privacy Amplification

Finally we perform privacy amplification, using hashing functions.

In [46]:
alice_digest = sha1(alice_correct_key)
print("Alice's correct key's digest is: {}".format(alice_digest))

bob_digest = sha1(bob_correct_key)
print("Bob's correct key's digest is: {}".format(bob_digest))

Alice's correct key's digest is: 11011100011000100100110010111101011110100111101111011100101111010010000000100011101110110001111111000010010101100100101001111011010100011011000000010100111011010110111011001000100011111011011100010010
Bob's correct key's digest is: 11011100011000100100110010111101011110100111101111011100101111010010000000100011101110110001111111000010010101100100101001111011010100011011000000010100111011010110111011001000100011111011011100010010
